In [1]:
import sys, math
import numpy as np

import Box2D
from Box2D.b2 import (edgeShape, circleShape, fixtureDef, polygonShape, revoluteJointDef, contactListener)

import gym
from gym import spaces
from gym.utils import colorize, seeding

from keras.models import Sequential
from keras.layers import Dense

/usr/local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# This is simple 4-joints walker robot environment.
#
# There are two versions:
#
# - Normal, with slightly uneven terrain.
#
# - Hardcore with ladders, stumps, pitfalls.
#
# Reward is given for moving forward, total 300+ points up to the far end. If the robot falls,
# it gets -100. Applying motor torque costs a small amount of points, more optimal agent
# will get better score.
#
# Heuristic is provided for testing, it's also useful to get demonstrations to
# learn from. To run heuristic:
#
# python gym/envs/box2d/bipedal_walker.py
#
# State consists of hull angle speed, angular velocity, horizontal speed, vertical speed,
# position of joints and joints angular speed, legs contact with ground, and 10 lidar
# rangefinder measurements to help to deal with the hardcore version. There's no coordinates
# in the state vector. Lidar is less useful in normal version, but it works.
#
# To solve the game you need to get 300 points in 1600 time steps.
#
# To solve hardcore version you need 300 points in 2000 time steps.
#
# Created by Oleg Klimov. Licensed on the same terms as the rest of OpenAI Gym.

FPS    = 50
SCALE  = 30.0   # affects how fast-paced the game is, forces should be adjusted as well

MOTORS_TORQUE = 120
SPEED_HIP     = 6
SPEED_KNEE    = 6
LIDAR_RANGE   = 160/SCALE

INITIAL_RANDOM = 5

HULL_POLY =[
    (-30,+9), (+6,+9), (+34,+1),
    (+34,-8), (-30,-8)
    ]
LEG_DOWN = -8/SCALE
LEG_W, LEG_H = 8/SCALE, 34/SCALE

VIEWPORT_W = 600
VIEWPORT_H = 400

TERRAIN_STEP   = 14/SCALE
TERRAIN_LENGTH = 200     # in steps
TERRAIN_HEIGHT = VIEWPORT_H/SCALE/4
TERRAIN_GRASS    = 10    # low long are grass spots, in steps
TERRAIN_STARTPAD = 20    # in steps
FRICTION = 2.5

HULL_FD = fixtureDef(
                shape=polygonShape(vertices=[ (x/SCALE,y/SCALE) for x,y in HULL_POLY ]),
                density=5.0,
                friction=0.1,
                categoryBits=0x0020,
                maskBits=0x001,  # collide only with ground
                restitution=0.0) # 0.99 bouncy

LEG_FD = fixtureDef(
                    shape=polygonShape(box=(LEG_W/2, LEG_H/2)),
                    density=1.0,
                    restitution=0.0,
                    categoryBits=0x0020,
                    maskBits=0x001)

LOWER_FD = fixtureDef(
                    shape=polygonShape(box=(0.8*LEG_W/2, LEG_H/2)),
                    density=1.0,
                    restitution=0.0,
                    categoryBits=0x0020,
                    maskBits=0x001)

In [3]:
class ContactDetector(contactListener):
    def __init__(self, env):
        contactListener.__init__(self)
        self.env = env
    def BeginContact(self, contact):
        if self.env.hull==contact.fixtureA.body or self.env.hull==contact.fixtureB.body:
            self.env.game_over = True
        for leg in [self.env.legs[1], self.env.legs[3]]:
            if leg in [contact.fixtureA.body, contact.fixtureB.body]:
                leg.ground_contact = True
    def EndContact(self, contact):
        for leg in [self.env.legs[1], self.env.legs[3]]:
            if leg in [contact.fixtureA.body, contact.fixtureB.body]:
                leg.ground_contact = False
def nnmodel(input_dim):
    model = Sequential()
    model.add(Dense(32, input_dim=input_dim, activation='relu'))
    model.add(Dense(16, activation='sigmoid'))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
    return model

In [4]:
class BipedalWalker(gym.Env):
    metadata = {
        'render.modes': ['human', 'rgb_array'],
        'video.frames_per_second' : FPS
    }

    hardcore = False

    def __init__(self):
        self.seed()
        self.viewer = None

        self.world = Box2D.b2World()
        self.terrain = None
        self.hull = None

        self.prev_shaping = None

        self.fd_polygon = fixtureDef(
                        shape = polygonShape(vertices=
                        [(0, 0),
                         (1, 0),
                         (1, -1),
                         (0, -1)]),
                        friction = FRICTION)

        self.fd_edge = fixtureDef(
                    shape = edgeShape(vertices=
                    [(0, 0),
                     (1, 1)]),
                    friction = FRICTION,
                    categoryBits=0x0001,
                )

        self.reset()

        high = np.array([np.inf]*24)
        self.action_space = spaces.Box(np.array([-1,-1,-1,-1]), np.array([+1,+1,+1,+1]))
        self.observation_space = spaces.Box(-high, high)

    def seed(self, seed=None):
        self.np_random, seed = seeding.np_random(seed)
        return [seed]

    def _destroy(self):
        if not self.terrain: return
        self.world.contactListener = None
        for t in self.terrain:
            self.world.DestroyBody(t)
        self.terrain = []
        self.world.DestroyBody(self.hull)
        self.hull = None
        for leg in self.legs:
            self.world.DestroyBody(leg)
        self.legs = []
        self.joints = []

    def _generate_terrain(self, hardcore):
        GRASS, STUMP, STAIRS, PIT, _STATES_ = range(5)
        state    = GRASS
        velocity = 0.0
        y        = TERRAIN_HEIGHT
        counter  = TERRAIN_STARTPAD
        oneshot  = False
        self.terrain   = []
        self.terrain_x = []
        self.terrain_y = []
        for i in range(TERRAIN_LENGTH):
            x = i*TERRAIN_STEP
            self.terrain_x.append(x)

            if state==GRASS and not oneshot:
                velocity = 0.8*velocity + 0.01*np.sign(TERRAIN_HEIGHT - y)
                if i > TERRAIN_STARTPAD: velocity += self.np_random.uniform(-1, 1)/SCALE   #1
                y += velocity

            elif state==PIT and oneshot:
                counter = self.np_random.randint(3, 5)
                poly = [
                    (x,              y),
                    (x+TERRAIN_STEP, y),
                    (x+TERRAIN_STEP, y-4*TERRAIN_STEP),
                    (x,              y-4*TERRAIN_STEP),
                    ]
                self.fd_polygon.shape.vertices=poly
                t = self.world.CreateStaticBody(
                    fixtures = self.fd_polygon)
                t.color1, t.color2 = (1,1,1), (0.6,0.6,0.6)
                self.terrain.append(t)

                self.fd_polygon.shape.vertices=[(p[0]+TERRAIN_STEP*counter,p[1]) for p in poly]
                t = self.world.CreateStaticBody(
                    fixtures = self.fd_polygon)
                t.color1, t.color2 = (1,1,1), (0.6,0.6,0.6)
                self.terrain.append(t)
                counter += 2
                original_y = y

            elif state==PIT and not oneshot:
                y = original_y
                if counter > 1:
                    y -= 4*TERRAIN_STEP

            elif state==STUMP and oneshot:
                counter = self.np_random.randint(1, 3)
                poly = [
                    (x,                      y),
                    (x+counter*TERRAIN_STEP, y),
                    (x+counter*TERRAIN_STEP, y+counter*TERRAIN_STEP),
                    (x,                      y+counter*TERRAIN_STEP),
                    ]
                self.fd_polygon.shape.vertices=poly
                t = self.world.CreateStaticBody(
                    fixtures = self.fd_polygon)
                t.color1, t.color2 = (1,1,1), (0.6,0.6,0.6)
                self.terrain.append(t)

            elif state==STAIRS and oneshot:
                stair_height = +1 if self.np_random.rand() > 0.5 else -1
                stair_width = self.np_random.randint(4, 5)
                stair_steps = self.np_random.randint(3, 5)
                original_y = y
                for s in range(stair_steps):
                    poly = [
                        (x+(    s*stair_width)*TERRAIN_STEP, y+(   s*stair_height)*TERRAIN_STEP),
                        (x+((1+s)*stair_width)*TERRAIN_STEP, y+(   s*stair_height)*TERRAIN_STEP),
                        (x+((1+s)*stair_width)*TERRAIN_STEP, y+(-1+s*stair_height)*TERRAIN_STEP),
                        (x+(    s*stair_width)*TERRAIN_STEP, y+(-1+s*stair_height)*TERRAIN_STEP),
                        ]
                    self.fd_polygon.shape.vertices=poly
                    t = self.world.CreateStaticBody(
                        fixtures = self.fd_polygon)
                    t.color1, t.color2 = (1,1,1), (0.6,0.6,0.6)
                    self.terrain.append(t)
                counter = stair_steps*stair_width

            elif state==STAIRS and not oneshot:
                s = stair_steps*stair_width - counter - stair_height
                n = s/stair_width
                y = original_y + (n*stair_height)*TERRAIN_STEP

            oneshot = False
            self.terrain_y.append(y)
            counter -= 1
            if counter==0:
                counter = self.np_random.randint(TERRAIN_GRASS/2, TERRAIN_GRASS)
                if state==GRASS and hardcore:
                    state = self.np_random.randint(1, _STATES_)
                    oneshot = True
                else:
                    state = GRASS
                    oneshot = True

        self.terrain_poly = []
        for i in range(TERRAIN_LENGTH-1):
            poly = [
                (self.terrain_x[i],   self.terrain_y[i]),
                (self.terrain_x[i+1], self.terrain_y[i+1])
                ]
            self.fd_edge.shape.vertices=poly
            t = self.world.CreateStaticBody(
                fixtures = self.fd_edge)
            color = (0.3, 1.0 if i%2==0 else 0.8, 0.3)
            t.color1 = color
            t.color2 = color
            self.terrain.append(t)
            color = (0.4, 0.6, 0.3)
            poly += [ (poly[1][0], 0), (poly[0][0], 0) ]
            self.terrain_poly.append( (poly, color) )
        self.terrain.reverse()

    def _generate_clouds(self):
        # Sorry for the clouds, couldn't resist
        self.cloud_poly   = []
        for i in range(TERRAIN_LENGTH//20):
            x = self.np_random.uniform(0, TERRAIN_LENGTH)*TERRAIN_STEP
            y = VIEWPORT_H/SCALE*3/4
            poly = [
                (x+15*TERRAIN_STEP*math.sin(3.14*2*a/5)+self.np_random.uniform(0,5*TERRAIN_STEP),
                 y+ 5*TERRAIN_STEP*math.cos(3.14*2*a/5)+self.np_random.uniform(0,5*TERRAIN_STEP) )
                for a in range(5) ]
            x1 = min( [p[0] for p in poly] )
            x2 = max( [p[0] for p in poly] )
            self.cloud_poly.append( (poly,x1,x2) )

    def reset(self):
        self._destroy()
        self.world.contactListener_bug_workaround = ContactDetector(self)
        self.world.contactListener = self.world.contactListener_bug_workaround
        self.game_over = False
        self.prev_shaping = None
        self.scroll = 0.0
        self.lidar_render = 0

        W = VIEWPORT_W/SCALE
        H = VIEWPORT_H/SCALE

        self._generate_terrain(self.hardcore)
        self._generate_clouds()

        init_x = TERRAIN_STEP*TERRAIN_STARTPAD/2
        init_y = TERRAIN_HEIGHT+2*LEG_H
        self.hull = self.world.CreateDynamicBody(
            position = (init_x, init_y),
            fixtures = HULL_FD
                )
        self.hull.color1 = (0.5,0.4,0.9)
        self.hull.color2 = (0.3,0.3,0.5)
        self.hull.ApplyForceToCenter((self.np_random.uniform(-INITIAL_RANDOM, INITIAL_RANDOM), 0), True)

        self.legs = []
        self.joints = []
        for i in [-1,+1]:
            leg = self.world.CreateDynamicBody(
                position = (init_x, init_y - LEG_H/2 - LEG_DOWN),
                angle = (i*0.05),
                fixtures = LEG_FD
                )
            leg.color1 = (0.6-i/10., 0.3-i/10., 0.5-i/10.)
            leg.color2 = (0.4-i/10., 0.2-i/10., 0.3-i/10.)
            rjd = revoluteJointDef(
                bodyA=self.hull,
                bodyB=leg,
                localAnchorA=(0, LEG_DOWN),
                localAnchorB=(0, LEG_H/2),
                enableMotor=True,
                enableLimit=True,
                maxMotorTorque=MOTORS_TORQUE,
                motorSpeed = i,
                lowerAngle = -0.8,
                upperAngle = 1.1,
                )
            self.legs.append(leg)
            self.joints.append(self.world.CreateJoint(rjd))

            lower = self.world.CreateDynamicBody(
                position = (init_x, init_y - LEG_H*3/2 - LEG_DOWN),
                angle = (i*0.05),
                fixtures = LOWER_FD
                )
            lower.color1 = (0.6-i/10., 0.3-i/10., 0.5-i/10.)
            lower.color2 = (0.4-i/10., 0.2-i/10., 0.3-i/10.)
            rjd = revoluteJointDef(
                bodyA=leg,
                bodyB=lower,
                localAnchorA=(0, -LEG_H/2),
                localAnchorB=(0, LEG_H/2),
                enableMotor=True,
                enableLimit=True,
                maxMotorTorque=MOTORS_TORQUE,
                motorSpeed = 1,
                lowerAngle = -1.6,
                upperAngle = -0.1,
                )
            lower.ground_contact = False
            self.legs.append(lower)
            self.joints.append(self.world.CreateJoint(rjd))

        self.drawlist = self.terrain + self.legs + [self.hull]

        class LidarCallback(Box2D.b2.rayCastCallback):
            def ReportFixture(self, fixture, point, normal, fraction):
                if (fixture.filterData.categoryBits & 1) == 0:
                    return 1
                self.p2 = point
                self.fraction = fraction
                return 0
        self.lidar = [LidarCallback() for _ in range(10)]

        return self.step(np.array([0,0,0,0]))[0]
    def set_state(self, state):
        return 0
    def step(self, action):
        self.hull.ApplyForceToCenter((0, 20), True) 
        control_speed = False  # Should be easier as well
        if control_speed:
            self.joints[0].motorSpeed = float(SPEED_HIP  * np.clip(action[0], -1, 1))
            self.joints[1].motorSpeed = float(SPEED_KNEE * np.clip(action[1], -1, 1))
            self.joints[2].motorSpeed = float(SPEED_HIP  * np.clip(action[2], -1, 1))
            self.joints[3].motorSpeed = float(SPEED_KNEE * np.clip(action[3], -1, 1))
        else:
            self.joints[0].motorSpeed     = float(SPEED_HIP     * np.sign(action[0]))
            self.joints[0].maxMotorTorque = float(MOTORS_TORQUE * np.clip(np.abs(action[0]), 0, 1))
            self.joints[1].motorSpeed     = float(SPEED_KNEE    * np.sign(action[1]))
            self.joints[1].maxMotorTorque = float(MOTORS_TORQUE * np.clip(np.abs(action[1]), 0, 1))
            self.joints[2].motorSpeed     = float(SPEED_HIP     * np.sign(action[2]))
            self.joints[2].maxMotorTorque = float(MOTORS_TORQUE * np.clip(np.abs(action[2]), 0, 1))
            self.joints[3].motorSpeed     = float(SPEED_KNEE    * np.sign(action[3]))
            self.joints[3].maxMotorTorque = float(MOTORS_TORQUE * np.clip(np.abs(action[3]), 0, 1))

        self.world.Step(1.0/FPS, 6*30, 2*30)

        pos = self.hull.position
        vel = self.hull.linearVelocity

        for i in range(10):
            self.lidar[i].fraction = 1.0
            self.lidar[i].p1 = pos
            self.lidar[i].p2 = (
                pos[0] + math.sin(1.5*i/10.0)*LIDAR_RANGE,
                pos[1] - math.cos(1.5*i/10.0)*LIDAR_RANGE)
            self.world.RayCast(self.lidar[i], self.lidar[i].p1, self.lidar[i].p2)

        state = [
            self.hull.angle,        # Normal angles up to 0.5 here, but sure more is possible.
            2.0*self.hull.angularVelocity/FPS,
            0.3*vel.x*(VIEWPORT_W/SCALE)/FPS,  # Normalized to get -1..1 range
            0.3*vel.y*(VIEWPORT_H/SCALE)/FPS,
            self.joints[0].angle,   # This will give 1.1 on high up, but it's still OK (and there should be spikes on hiting the ground, that's normal too)
            self.joints[0].speed / SPEED_HIP,
            self.joints[1].angle + 1.0,
            self.joints[1].speed / SPEED_KNEE,
            1.0 if self.legs[1].ground_contact else 0.0,
            self.joints[2].angle,
            self.joints[2].speed / SPEED_HIP,
            self.joints[3].angle + 1.0,
            self.joints[3].speed / SPEED_KNEE,
            1.0 if self.legs[3].ground_contact else 0.0
            ]
        state += [l.fraction for l in self.lidar]
        assert len(state)==24

        self.scroll = pos.x - VIEWPORT_W/SCALE/5

        shaping  = 500*pos[0]/SCALE   # moving forward is a way to receive reward (normalized to get 300 on completion)
        shaping -= 5.0*abs(state[0])  # keep head straight, other than that and falling, any behavior is unpunished

        reward = 0
        if self.prev_shaping is not None:
            reward = shaping - self.prev_shaping
        self.prev_shaping = shaping

        for a in action:
            reward -= 0.00015 * MOTORS_TORQUE * np.clip(np.abs(a), 0, 1)
            # normalized to about -50.0 using heuristic, more optimal agent should spend less

        done = False
        if self.game_over or pos[0] < 0:
            reward = -100
            done   = True
        if pos[0] > (TERRAIN_LENGTH-TERRAIN_GRASS)*TERRAIN_STEP:
            done   = True
        return np.array(state), reward, done, {}

    def render(self, mode='human'):
        from gym.envs.classic_control import rendering
        if self.viewer is None:
            self.viewer = rendering.Viewer(VIEWPORT_W, VIEWPORT_H)
        self.viewer.set_bounds(self.scroll, VIEWPORT_W/SCALE + self.scroll, 0, VIEWPORT_H/SCALE)

        self.viewer.draw_polygon( [
            (self.scroll,                  0),
            (self.scroll+VIEWPORT_W/SCALE, 0),
            (self.scroll+VIEWPORT_W/SCALE, VIEWPORT_H/SCALE),
            (self.scroll,                  VIEWPORT_H/SCALE),
            ], color=(0.9, 0.9, 1.0) )
        for poly,x1,x2 in self.cloud_poly:
            if x2 < self.scroll/2: continue
            if x1 > self.scroll/2 + VIEWPORT_W/SCALE: continue
            self.viewer.draw_polygon( [(p[0]+self.scroll/2, p[1]) for p in poly], color=(1,1,1))
        for poly, color in self.terrain_poly:
            if poly[1][0] < self.scroll: continue
            if poly[0][0] > self.scroll + VIEWPORT_W/SCALE: continue
            self.viewer.draw_polygon(poly, color=color)

        self.lidar_render = (self.lidar_render+1) % 100
        i = self.lidar_render
        if i < 2*len(self.lidar):
            l = self.lidar[i] if i < len(self.lidar) else self.lidar[len(self.lidar)-i-1]
            self.viewer.draw_polyline( [l.p1, l.p2], color=(1,0,0), linewidth=1 )

        for obj in self.drawlist:
            for f in obj.fixtures:
                trans = f.body.transform
                if type(f.shape) is circleShape:
                    t = rendering.Transform(translation=trans*f.shape.pos)
                    self.viewer.draw_circle(f.shape.radius, 30, color=obj.color1).add_attr(t)
                    self.viewer.draw_circle(f.shape.radius, 30, color=obj.color2, filled=False, linewidth=2).add_attr(t)
                else:
                    path = [trans*v for v in f.shape.vertices]
                    self.viewer.draw_polygon(path, color=obj.color1)
                    path.append(path[0])
                    self.viewer.draw_polyline(path, color=obj.color2, linewidth=2)

        flagy1 = TERRAIN_HEIGHT
        flagy2 = flagy1 + 50/SCALE
        x = TERRAIN_STEP*3
        self.viewer.draw_polyline( [(x, flagy1), (x, flagy2)], color=(0,0,0), linewidth=2 )
        f = [(x, flagy2), (x, flagy2-10/SCALE), (x+25/SCALE, flagy2-5/SCALE)]
        self.viewer.draw_polygon(f, color=(0.9,0.2,0) )
        self.viewer.draw_polyline(f + [f[0]], color=(0,0,0), linewidth=2 )

        return self.viewer.render(return_rgb_array = mode=='rgb_array')

    def close(self):
        if self.viewer is not None:
            self.viewer.close()
            self.viewer = None

In [5]:
class BipedalWalkerHardcore(BipedalWalker):
    hardcore = True

In [6]:
# if __name__=="__main__":
#     env = BipedalWalker()
#     env.reset()
#     steps = 0
#     total_reward = 0
#     a = np.array([0.0, 0.0, 0.0, 0.0])
#     STAY_ON_ONE_LEG, PUT_OTHER_DOWN, PUSH_OFF = 1,2,3
#     SPEED = 0.29  # Will fall forward on higher speed
#     state = STAY_ON_ONE_LEG
#     moving_leg = 0
#     supporting_leg = 1 - moving_leg
#     SUPPORT_KNEE_ANGLE = +0.1
#     supporting_knee_angle = SUPPORT_KNEE_ANGLE
#     while True:
#         s, r, done, info = env.step(a)
#         total_reward += r
#         if steps % 20 == 0 or done:
#             print("\naction " + str(["{:+0.2f}".format(x) for x in a]))
#             print("state vector: ",s)
#             print("step {} total_reward {:+0.2f}".format(steps, total_reward))
#             print("hull " + str(["{:+0.2f}".format(x) for x in s[0:4] ]))
#             print("leg0 " + str(["{:+0.2f}".format(x) for x in s[4:9] ]))
#             print("leg1 " + str(["{:+0.2f}".format(x) for x in s[9:14]]))
#         steps += 1

#         contact0 = s[8]
#         contact1 = s[13]
#         moving_s_base = 4 + 5*moving_leg
#         supporting_s_base = 4 + 5*supporting_leg

#         hip_targ  = [None,None]   # -0.8 .. +1.1
#         knee_targ = [None,None]   # -0.6 .. +0.9
#         hip_todo  = [0.0, 0.0]
#         knee_todo = [0.0, 0.0]

#         if state==STAY_ON_ONE_LEG:
#             hip_targ[moving_leg]  = 1.1
#             knee_targ[moving_leg] = -0.6
#             supporting_knee_angle += 0.03
#             if s[2] > SPEED: supporting_knee_angle += 0.03
#             supporting_knee_angle = min( supporting_knee_angle, SUPPORT_KNEE_ANGLE )
#             knee_targ[supporting_leg] = supporting_knee_angle
#             if s[supporting_s_base+0] < 0.10: # supporting leg is behind
#                 state = PUT_OTHER_DOWN
#         if state==PUT_OTHER_DOWN:
#             hip_targ[moving_leg]  = +0.1
#             knee_targ[moving_leg] = SUPPORT_KNEE_ANGLE
#             knee_targ[supporting_leg] = supporting_knee_angle
#             if s[moving_s_base+4]:
#                 state = PUSH_OFF
#                 supporting_knee_angle = min( s[moving_s_base+2], SUPPORT_KNEE_ANGLE )
#         if state==PUSH_OFF:
#             knee_targ[moving_leg] = supporting_knee_angle
#             knee_targ[supporting_leg] = +1.0
#             if s[supporting_s_base+2] > 0.88 or s[2] > 1.2*SPEED:
#                 state = STAY_ON_ONE_LEG
#                 moving_leg = 1 - moving_leg
#                 supporting_leg = 1 - moving_leg

#         if hip_targ[0]: hip_todo[0] = 0.9*(hip_targ[0] - s[4]) - 0.25*s[5]
#         if hip_targ[1]: hip_todo[1] = 0.9*(hip_targ[1] - s[9]) - 0.25*s[10]
#         if knee_targ[0]: knee_todo[0] = 4.0*(knee_targ[0] - s[6])  - 0.25*s[7]
#         if knee_targ[1]: knee_todo[1] = 4.0*(knee_targ[1] - s[11]) - 0.25*s[12]

#         hip_todo[0] -= 0.9*(0-s[0]) - 1.5*s[1] # PID to keep head strait
#         hip_todo[1] -= 0.9*(0-s[0]) - 1.5*s[1]
#         knee_todo[0] -= 15.0*s[3]  # vertical speed, to damp oscillations
#         knee_todo[1] -= 15.0*s[3]

#         a[0] = hip_todo[0]
#         a[1] = knee_todo[0]
#         a[2] = hip_todo[1]
#         a[3] = knee_todo[1]
        
#         a = np.clip(0.5*a, -1.0, 1.0)

#         env.render()
#         if done: break

In [7]:
# let's build a random forest regressor
# feature vector would be state space and randonmized actions 
# label vector is the reward of that state space together with the randomized actions

In [8]:
X_train = []
y_train = []

In [ ]:
import random
from sklearn.ensemble import RandomForestRegressor
from sklearn import linear_model
from sklearn.linear_model import ElasticNet
if __name__=="__main__":
    env = BipedalWalker()
    env.reset()
    steps = 0
    total_reward = 0
    a = np.array([0.0, 0.0, 0.0, 0.0])
    SPEED = 0.29  # Will fall forward on higher speed
    prev_s = list(np.array(random.sample(range(-100, 100), 24))/100)
    modelTrained = False
#     model = RandomForestRegressor(max_depth = 5, n_estimators=50)
#     model = linear_model.SGDRegressor()
#     model = ElasticNet(random_state=0)
    model = nnmodel(28)

    tr = 0 
    prev_r = 0
    while steps <= 100000:
        
        new_s, r, done, info = env.step(a)      
        X_train.append(list(prev_s)+list(a))
        y_train.append(r)
        
        if steps > 2000 and steps %100 ==0:
            # re-train a model 
            print("training model model")
            modelTrained = True
            model.fit(np.array(X_train),np.array(y_train).reshape(len(y_train),1), epochs = 10, batch_size=20)
            
        if modelTrained:
            maxr = -1000
            maxa = None
            for i in range(100):
                h1 = np.random.randint(-1000,1000)/1000
                k1 = np.random.randint(-1000,1000)/1000
                h2 = np.random.randint(-1000,1000)/1000
                k2 = np.random.randint(-1000,1000)/1000
                
                if env.joints[0].angle >0:
                    if h1 >0:
                        h1 = -1 * h1
                else:
                    if h1 <0:
                        h1 = -1 * h1
                    
                if env.joints[2].angle >0:
                    if h2>0:
                        h2 = -1 * h2
                else:
                    if h2<0:
                        h2 = -1 * h2
                    
                testa = (h1,\
                 k1,\
                 h2,\
                 k2)
                
                r_pred = model.predict(np.array(list(new_s)+list(testa)).reshape(1,len(list(new_s)+list(testa))))
                if r_pred > maxr:
                    maxr = r_pred
                    maxa = testa
            a = maxa
            
        else:
            a = (np.random.randint(-1000,1000)/1000,\
                 np.random.randint(-1000,1000)/1000,\
                 np.random.randint(-1000,1000)/1000,\
                 np.random.randint(-1000,1000)/1000)
    
        if steps %100 == 0:
            
            print("At step ", steps)
            print("total rewards ", tr)
        prev_s = new_s
        prev_r = r
        if tr < -5:
            env.reset()
            tr = 0 
        # train the random forest every 1000 iterations:
        tr = tr + r
        
        steps += 1
        env.render()


At step  0
total rewards  0
At step  100
total rewards  -1.6461058749628814
At step  200
total rewards  -4.889282763146148
At step  300
total rewards  11.271799476111863
At step  400
total rewards  0.634659855709842
At step  500
total rewards  -1.0502854771050667
At step  600
total rewards  2.646164595161602
At step  700
total rewards  -1.3651118631836918
At step  800
total rewards  -0.6713333165491222
At step  900
total rewards  -1.3901378641299156
At step  1000
total rewards  12.967087756214276
At step  1100
total rewards  -3.000662841225102
At step  1200
total rewards  -0.9450915104130211
At step  1300
total rewards  -3.288754871509655
At step  1400
total rewards  -1.4296271829696894
At step  1500
total rewards  -1.1843130761428473
At step  1600
total rewards  -1.8130222440560904
At step  1700
total rewards  -4.104577005108383
At step  1800
total rewards  -1.2610322043322624
At step  1900
total rewards  -1.5507333134470584
At step  2000
total rewards  -3.616242658422261
training mod

2701/2701 [==============================] - 0s 70us/step - loss: 55.2202 - acc: 0.0000e+00
Epoch 7/10
2701/2701 [==============================] - 0s 73us/step - loss: 54.5936 - acc: 0.0000e+00
Epoch 8/10
2701/2701 [==============================] - 0s 74us/step - loss: 54.0819 - acc: 0.0000e+00
Epoch 9/10
2701/2701 [==============================] - 0s 77us/step - loss: 53.6132 - acc: 0.0000e+00
Epoch 10/10
2701/2701 [==============================] - 0s 71us/step - loss: 53.2256 - acc: 0.0000e+00
At step  2700
total rewards  -3.015784271105259
training model model
Epoch 1/10
2801/2801 [==============================] - 0s 78us/step - loss: 57.9718 - acc: 0.0000e+00
Epoch 2/10
2801/2801 [==============================] - 0s 83us/step - loss: 57.6142 - acc: 0.0000e+00
Epoch 3/10
2801/2801 [==============================] - 0s 77us/step - loss: 57.1328 - acc: 0.0000e+00
Epoch 4/10
2801/2801 [==============================] - 0s 74us/step - loss: 56.7634 - acc: 0.0000e+00
Epoch 5/10
280

3401/3401 [==============================] - 0s 110us/step - loss: 32.3640 - acc: 0.0000e+00
At step  3400
total rewards  -0.7858283032417297
training model model
Epoch 1/10
3501/3501 [==============================] - 0s 89us/step - loss: 30.5378 - acc: 0.0000e+00
Epoch 2/10
3501/3501 [==============================] - 0s 76us/step - loss: 30.3637 - acc: 0.0000e+00
Epoch 3/10
3501/3501 [==============================] - 0s 74us/step - loss: 30.1708 - acc: 0.0000e+00
Epoch 4/10
3501/3501 [==============================] - 0s 71us/step - loss: 30.0656 - acc: 0.0000e+00
Epoch 5/10
3501/3501 [==============================] - 0s 78us/step - loss: 29.8198 - acc: 0.0000e+00
Epoch 6/10
3501/3501 [==============================] - 0s 76us/step - loss: 29.7852 - acc: 0.0000e+00
Epoch 7/10
3501/3501 [==============================] - 0s 72us/step - loss: 30.6271 - acc: 0.0000e+00
Epoch 8/10
3501/3501 [==============================] - 0s 76us/step - loss: 29.5069 - acc: 0.0000e+00
Epoch 9/10
35

Epoch 4/10
4201/4201 [==============================] - 1s 121us/step - loss: 23.4704 - acc: 0.0000e+00
Epoch 5/10
4201/4201 [==============================] - 0s 86us/step - loss: 23.4469 - acc: 0.0000e+00
Epoch 6/10
4201/4201 [==============================] - 0s 89us/step - loss: 22.8511 - acc: 0.0000e+00
Epoch 7/10
4201/4201 [==============================] - 0s 85us/step - loss: 22.5055 - acc: 0.0000e+00
Epoch 8/10
4201/4201 [==============================] - 0s 87us/step - loss: 22.3684 - acc: 0.0000e+00
Epoch 9/10
4201/4201 [==============================] - 0s 93us/step - loss: 23.4251 - acc: 0.0000e+00
Epoch 10/10
4201/4201 [==============================] - 1s 128us/step - loss: 21.7185 - acc: 0.0000e+00
At step  4200
total rewards  -2.792928257564664
training model model
Epoch 1/10
4301/4301 [==============================] - 0s 88us/step - loss: 21.1459 - acc: 0.0000e+00
Epoch 2/10
4301/4301 [==============================] - 0s 101us/step - loss: 21.4832 - acc: 0.0000e+00


4901/4901 [==============================] - 0s 76us/step - loss: 16.8439 - acc: 0.0000e+00
Epoch 9/10
4901/4901 [==============================] - 0s 74us/step - loss: 15.9549 - acc: 0.0000e+00
Epoch 10/10
4901/4901 [==============================] - 0s 79us/step - loss: 15.9040 - acc: 0.0000e+00
At step  4900
total rewards  -0.3419941158918368
training model model
Epoch 1/10
5001/5001 [==============================] - 0s 80us/step - loss: 21.7120 - acc: 0.0000e+00
Epoch 2/10
5001/5001 [==============================] - 0s 77us/step - loss: 18.6539 - acc: 0.0000e+00
Epoch 3/10
5001/5001 [==============================] - 0s 77us/step - loss: 17.8323 - acc: 0.0000e+00
Epoch 4/10
5001/5001 [==============================] - 0s 78us/step - loss: 17.7435 - acc: 0.0000e+00
Epoch 5/10
5001/5001 [==============================] - 0s 76us/step - loss: 16.9542 - acc: 0.0000e+00
Epoch 6/10
5001/5001 [==============================] - 0s 74us/step - loss: 16.9226 - acc: 0.0000e+00
Epoch 7/10
50

...

42701/42701 [==============================] - 2s 48us/step - loss: 23.1213 - acc: 0.0000e+00
Epoch 6/10
42701/42701 [==============================] - 2s 49us/step - loss: 25.9657 - acc: 0.0000e+00
Epoch 7/10
42701/42701 [==============================] - 2s 48us/step - loss: 23.7463 - acc: 0.0000e+00
Epoch 8/10
42701/42701 [==============================] - 2s 50us/step - loss: 24.2661 - acc: 0.0000e+00
Epoch 9/10
42701/42701 [==============================] - 2s 49us/step - loss: 23.2078 - acc: 0.0000e+00
Epoch 10/10
42701/42701 [==============================] - 2s 49us/step - loss: 24.0303 - acc: 0.0000e+00
At step  42700
total rewards  -1.486107266203359
training model model
Epoch 1/10
42801/42801 [==============================] - 2s 49us/step - loss: 24.8009 - acc: 0.0000e+00
Epoch 2/10
42801/42801 [==============================] - 2s 48us/step - loss: 24.2975 - acc: 0.0000e+00
Epoch 3/10
42801/42801 [==============================] - 2s 49us/step - loss: 24.7561 - acc: 0.0000

43401/43401 [==============================] - 2s 49us/step - loss: 24.0374 - acc: 0.0000e+00
Epoch 9/10
43401/43401 [==============================] - 2s 49us/step - loss: 23.6242 - acc: 0.0000e+00
Epoch 10/10
43401/43401 [==============================] - 2s 49us/step - loss: 24.9779 - acc: 0.0000e+00
At step  43400
total rewards  -1.0995246046235265
training model model
Epoch 1/10
43501/43501 [==============================] - 2s 51us/step - loss: 25.6521 - acc: 0.0000e+00
Epoch 2/10
43501/43501 [==============================] - 2s 50us/step - loss: 24.6090 - acc: 0.0000e+00
Epoch 3/10
43501/43501 [==============================] - 2s 49us/step - loss: 27.2690 - acc: 0.0000e+00
Epoch 4/10
43501/43501 [==============================] - 2s 49us/step - loss: 25.7988 - acc: 0.0000e+00
Epoch 5/10
43501/43501 [==============================] - 2s 49us/step - loss: 24.5422 - acc: 0.0000e+00
Epoch 6/10
43501/43501 [==============================] - 2s 49us/step - loss: 25.6571 - acc: 0.000

training model model
Epoch 1/10
44201/44201 [==============================] - 2s 49us/step - loss: 26.0011 - acc: 0.0000e+00
Epoch 2/10
44201/44201 [==============================] - 2s 49us/step - loss: 27.1720 - acc: 0.0000e+00
Epoch 3/10
44201/44201 [==============================] - 2s 49us/step - loss: 24.8254 - acc: 0.0000e+00
Epoch 4/10
44201/44201 [==============================] - 2s 49us/step - loss: 27.4567 - acc: 0.0000e+00
Epoch 5/10
44201/44201 [==============================] - 2s 49us/step - loss: 24.8464 - acc: 0.0000e+00
Epoch 6/10
44201/44201 [==============================] - 2s 49us/step - loss: 24.9864 - acc: 0.0000e+00
Epoch 7/10
44201/44201 [==============================] - 2s 49us/step - loss: 26.2044 - acc: 0.0000e+00
Epoch 8/10
44201/44201 [==============================] - 2s 49us/step - loss: 25.2366 - acc: 0.0000e+00
Epoch 9/10
44201/44201 [==============================] - 2s 49us/step - loss: 25.4909 - acc: 0.0000e+00
Epoch 10/10
44201/44201 [=========

44901/44901 [==============================] - 6s 128us/step - loss: 25.0485 - acc: 0.0000e+00
Epoch 4/10
44901/44901 [==============================] - 6s 139us/step - loss: 26.2186 - acc: 0.0000e+00
Epoch 5/10
44901/44901 [==============================] - 6s 136us/step - loss: 25.4961 - acc: 0.0000e+00
Epoch 6/10
44901/44901 [==============================] - 6s 127us/step - loss: 25.3272 - acc: 0.0000e+00
Epoch 7/10
44901/44901 [==============================] - 6s 128us/step - loss: 24.6716 - acc: 0.0000e+00
Epoch 8/10
44901/44901 [==============================] - 6s 127us/step - loss: 25.3306 - acc: 0.0000e+00
Epoch 9/10
44901/44901 [==============================] - 6s 127us/step - loss: 25.9991 - acc: 0.0000e+00
Epoch 10/10
44901/44901 [==============================] - 6s 127us/step - loss: 25.3527 - acc: 0.0000e+00
At step  44900
total rewards  -0.7852463724862045
training model model
Epoch 1/10
45001/45001 [==============================] - 6s 127us/step - loss: 26.1893 - a

45601/45601 [==============================] - 6s 128us/step - loss: 25.7801 - acc: 0.0000e+00
Epoch 6/10
45601/45601 [==============================] - 6s 128us/step - loss: 25.1649 - acc: 0.0000e+00
Epoch 7/10
45601/45601 [==============================] - 6s 128us/step - loss: 26.3019 - acc: 0.0000e+00
Epoch 8/10
45601/45601 [==============================] - 6s 128us/step - loss: 25.2193 - acc: 0.0000e+00
Epoch 9/10
45601/45601 [==============================] - 6s 128us/step - loss: 26.7884 - acc: 0.0000e+00
Epoch 10/10
45601/45601 [==============================] - 6s 128us/step - loss: 25.1324 - acc: 0.0000e+00
At step  45600
total rewards  -0.9625950992991923
training model model
Epoch 1/10
45701/45701 [==============================] - 6s 128us/step - loss: 26.2014 - acc: 0.0000e+00
Epoch 2/10
45701/45701 [==============================] - 6s 129us/step - loss: 24.9420 - acc: 0.0000e+00
Epoch 3/10
45701/45701 [==============================] - 6s 131us/step - loss: 25.3790 - a